In [ ]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Setări directoare
train_dir = '/kaggle/input/car-vs-bike-classification-dataset/Car-Bike-Dataset'
test_dir = train_dir  # dacă vrei să folosești tot același set pentru test, altfel setează separat

In [ ]:
# Vizualizare imagini random din set
def show_random_images(base_dir, num_images=5):
    image_paths = []
    for class_name in os.listdir(base_dir):
        class_dir = os.path.join(base_dir, class_name)
        if os.path.isdir(class_dir):
            for fname in os.listdir(class_dir):
                if fname.lower().endswith(('jpg', 'jpeg', 'png')):
                    image_paths.append(os.path.join(class_dir, fname))
    sample_files = random.sample(image_paths, num_images)
    plt.figure(figsize=(15, 5))
    for idx, f in enumerate(sample_files):
        img = Image.open(f)
        class_name = os.path.basename(os.path.dirname(f))
        plt.subplot(1, num_images, idx + 1)
        plt.imshow(img)
        plt.title(class_name)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

show_random_images(train_dir)

In [ ]:
# split subfolders Car/Bike
file_paths = []
labels = []
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    if not os.path.isdir(class_dir):
        continue
    for fname in os.listdir(class_dir):
        if fname.lower().endswith(('jpg', 'jpeg', 'png')):
            file_paths.append(os.path.join(class_dir, fname))
            labels.append(0 if class_name.lower() == 'bike' else 1)  # 0: bike, 1: car

train_paths, val_paths, train_labels, val_labels = train_test_split(
    file_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

In [ ]:
class CarBikeDataset(Dataset):
    def __init__(self, file_paths, labels=None, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.file_paths)
    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        else:
            return image, os.path.basename(img_path)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = CarBikeDataset(train_paths, train_labels, transform)
val_dataset = CarBikeDataset(val_paths, val_labels, transform)
# test_set (same director) // aici pot inlocui cu folder de test 
test_file_paths = []
for root, _, files in os.walk(test_dir):
    for f in files:
        if f.lower().endswith(('jpg', 'jpeg', 'png')):
            test_file_paths.append(os.path.join(root, f))
test_dataset = CarBikeDataset(test_file_paths, labels=None, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

aici creez un model cu 3 straturi convolutionale +pooling pt extragerea de features 
flatten -> 3D to vector 
Linear(128 * 28 * 28, 512)" este un strat foarte mare

In [ ]:
class CarBikeCNN(nn.Module):
    def __init__(self):
        super(CarBikeCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            # last layer - 2 neuroni pentru clasificare binara 
            nn.Linear(512, 2)  # car / bike
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CarBikeCNN().to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {count_parameters(model):,} number of trainable parameters.")

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 5
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs):
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    for epoch in range(epochs):
        model.train()
        running_loss, correct = 0.0, 0
        train_loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)
        for images, labels in train_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            train_loop.set_postfix(loss=loss.item())
        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / len(train_loader.dataset)
        model.eval()
        val_loss, val_correct = 0.0, 0
        val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
        with torch.no_grad():
            for images, labels in val_loop:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                val_correct += (outputs.argmax(1) == labels).sum().item()
                val_loop.set_postfix(val_loss=loss.item())
        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_acc)
        val_accuracies.append(val_acc)
        print(f"Epoch {epoch+1}/{epochs} => Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")
    return train_losses, val_losses, train_accuracies, val_accuracies

In [ ]:
train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    model, train_loader, val_loader, criterion, optimizer, device, epochs=5
)

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.legend()
plt.title("Loss")
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Acc')
plt.plot(val_accuracies, label='Val Acc')
plt.legend()
plt.title("Accuracy")
plt.show()

In [ ]:
def plot_top_misclassified(model, dataloader, class_names=['bike', 'car'], device='cuda', top_n=5):
    model.eval()
    misclassified = []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            mis_idx = preds != labels
            for i in range(images.size(0)):
                if mis_idx[i]:
                    misclassified.append((images[i].cpu(), preds[i].item(), labels[i].item()))
    misclassified_bike = [x for x in misclassified if x[2] == 0][:top_n]
    misclassified_car = [x for x in misclassified if x[2] == 1][:top_n]
    def show_images(examples, title):
        fig, axes = plt.subplots(1, len(examples), figsize=(4 * len(examples), 4))
        fig.suptitle(title, fontsize=16)
        if len(examples) == 1: axes = [axes]
        for ax, (img_tensor, pred, true) in zip(axes, examples):
            img = img_tensor.permute(1, 2, 0).numpy()
            img = (img * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406])
            img = np.clip(img, 0, 1)
            ax.imshow(img)
            ax.set_title(f'Pred: {class_names[pred]}\nTrue: {class_names[true]}', fontsize=12)
            ax.axis('off')
        plt.tight_layout()
        plt.show()
    show_images(misclassified_bike, 'Top Misclassified Bike')
    show_images(misclassified_car, 'Top Misclassified Car')

In [ ]:
class_names = ['bike', 'car']
plot_top_misclassified(model, val_loader, class_names, device)

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/cnn_car_bike_model_weights.pth')
torch.save(model, '/kaggle/working/cnn_car_bike_full_model.pth')